In [1]:
import urllib
import os
import zipfile

import numpy as np

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Conjunto de Dados

In [5]:
train = pd.read_csv("data/reviews_train.csv")
test = pd.read_csv("data/reviews_test.csv")

## Análise de Sentimentos

Nós estamos interessados em predizer o sentimento das revisões a partir de seu contéudo. Portanto, devemos construir um modelo que seja capaz de classificar o sentimento (positivo ou negativo) usando o texto.

In [26]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer

#Configuração para separação dos dados de treino
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

#Extração de features baseadas no texto
tfidf = TfidfVectorizer(stop_words = 'english', norm='l2')
X = tfidf.fit_transform(train["review"].values).toarray()
y = train["sentiment"].values

model = LinearSVC()
for X_train, y_train in sss.split(X, y):
    model.fit(X[X_train], y[y_train])
    print(model.score())

ValueError: Found input variables with inconsistent numbers of samples: [2800, 700]

In [22]:
tfidf_matrix

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [25]:
train["review"].shape

(3500,)